In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp bbox_video_annotator

In [ ]:
# hide
from nbdev import *

In [ ]:
#exporti
import attr
import warnings
from pubsub import pub
from attr import asdict
from copy import deepcopy
from typing import List, Dict, Optional
from itertools import groupby
from ipywidgets import HBox, VBox, Layout, Button, Checkbox, ToggleButton
from ipyannotator.bbox_canvas import BBoxVideoCanvas
from ipyannotator.base import AppWidgetState
from ipyannotator.right_menu_widget import BBoxVideoList, BBoxVideoItem
from ipyannotator.mltypes import BboxCoordinate, BboxVideoCoordinate
from ipyannotator.bbox_annotator import (
    BBoxAnnotator, BBoxState, BBoxAnnotatorController,
    BBoxAnnotatorGUI, BBoxCoordinates, BBoxCanvasState
)
from ipyannotator.services.bbox_trajectory import BBoxTrajectory

In [ ]:
# hide
import ipytest
import pytest
ipytest.autoconfig(raise_on_error=True)

In [ ]:
#exporti

@attr.define
class BboxVideoCoordSelected:
    index: int
    frame: int
    bbox_video_coordinate: BboxVideoCoordinate

In [ ]:
#exporti

@attr.define
class BboxVideoHistory:
    label: List[str] = []
    trajectory: Dict[str, List[BboxCoordinate]] = {}
    bbox_coord: Optional[BboxVideoCoordinate] = None

In [ ]:
#exporti

class BBoxVideoState(BBoxState):
    trajectory: Dict[str, List[BboxCoordinate]] = {}
    drawing_trajectory_enabled: bool = True
    right_menu_enabled: bool = True
    bbox_coords_selected: List[BboxVideoCoordSelected] = []
    merged_trajectories: List[str] = []

In [ ]:
#exporti

class BBoxVideoCoordinates(BBoxCoordinates):
    """Connects the BBoxList and the states"""

    def __init__(
        self,
        app_state: AppWidgetState,
        bbox_canvas_state: BBoxCanvasState,
        bbox_state: BBoxVideoState,
        drawing_enabled: bool,
        on_btn_select_clicked: callable = None,
        on_label_changed: callable = None,
        on_trajectory_enabled_clicked=None,
    ):
        super().__init__(
            app_state,
            bbox_canvas_state,
            bbox_state,
            on_btn_select_clicked
        )

        self.trajectory_enabled_checkbox = Checkbox(
            description='Enable object tracking',
            value=self._bbox_state.drawing_trajectory_enabled
        )
        if on_trajectory_enabled_clicked:
            self.trajectory_enabled_checkbox.observe(on_trajectory_enabled_clicked, names='value')

        pub.unsubscribe(super()._sync_labels, f'{bbox_canvas_state.root_topic}.bbox_coords')
        pub.unsubscribe(super()._refresh_children, f'{app_state.root_topic}.index')

        self._bbox_list = BBoxVideoList(
            btn_delete_enabled=drawing_enabled,
            on_label_changed=on_label_changed,
            on_btn_delete_clicked=self.on_btn_delete_clicked,
            on_btn_select_clicked=on_btn_select_clicked,
            classes=bbox_state.classes,
            on_checkbox_object_clicked=self._on_checkbox_object_clicked
        )

        bbox_canvas_state.subscribe(self._update_max_coord_input, 'image_scale')

        self.children = self._bbox_list.children

    def __getitem__(self, index: int) -> BBoxVideoItem:
        return self.children[1][index]

    def _render(self, bbox_coords: List[BboxVideoCoordinate], labels: List[List[str]]):
        if self._bbox_state.right_menu_enabled:
            selected = []
            all_frame_object_ids = [bbox_coord.id for bbox_coord in self._bbox_state.coords]

            for coord in self._bbox_state.bbox_coords_selected:
                if coord.bbox_video_coordinate.id in all_frame_object_ids:
                    selected.append(coord.index)

            self._bbox_list.render_btn_list(
                bbox_coords=bbox_coords,
                classes=labels,
                labels=self._bbox_state.labels,
                selected=selected
            )
            self.children = VBox([
                self.trajectory_enabled_checkbox,
                self._bbox_list,
            ]).children

    def on_btn_delete_clicked(self, index: int):
        super().on_btn_delete_clicked(index)
        self._remove_object_selected(index)

    def _remove_object_selected(self, index: int):
        try:
            self._bbox_state.set_quietly(
                'bbox_coords_selected',
                list(filter(lambda x: x.index != index, self._bbox_state.bbox_coords_selected))
            )
        except Exception:
            warnings.warn("Couldn't unselect object")

    def _update_max_coord_input(self, image_scale: float):
        """CoordinateInput maximum values that a user
        can change. 'x' and 'y' can be improved to avoid
        bbox outside of the canvas area."""

        self._bbox_list.max_coord_input_values = {
            'x': self._app_state.size[0] / image_scale,
            'y': self._app_state.size[1] / image_scale,
            'width': self._app_state.size[0] / image_scale,
            'height': self._app_state.size[1] / image_scale
        }

    def _on_checkbox_object_clicked(self, change: dict, index: int,
                                    bbox_coord: BboxVideoCoordinate):
        if change['new']:
            self._bbox_state.bbox_coords_selected.append(
                BboxVideoCoordSelected(
                    frame=self._app_state.index,
                    index=index,
                    bbox_video_coordinate=bbox_coord
                )
            )
        else:
            self._remove_object_selected(index)

In [ ]:
#exporti

class BBoxAnnotatorVideoGUI(BBoxAnnotatorGUI):
    def __init__(
        self,
        on_label_changed: callable,
        on_join_btn_clicked: callable,
        on_bbox_drawn: callable,
        *args,
        drawing_enabled: bool = True,
        **kwargs
    ):
        super().__init__(*args, **kwargs)

        self.on_bbox_drawn = on_bbox_drawn
        self.bbox_trajectory = BBoxTrajectory()
        self.history = BboxVideoHistory()

        pub.unsubAll(f'{self._image_box.state.root_topic}.bbox_coords')

        self._image_box = BBoxVideoCanvas(
            *self._app_state.size,
            drawing_enabled=drawing_enabled
        )

        self.right_menu = BBoxVideoCoordinates(
            app_state=self._app_state,
            bbox_canvas_state=self._image_box.state,
            bbox_state=self._bbox_state,
            on_btn_select_clicked=self._highlight_bbox,
            on_label_changed=on_label_changed,
            drawing_enabled=drawing_enabled,
            on_trajectory_enabled_clicked=self.on_trajectory_enabled_clicked
        )

        self._navi.on_navi_clicked = self.view_idx_changed

        self.center = HBox(
            [self._image_box, self.right_menu],
            layout=Layout(
                display='flex',
                flex_flow='row'
            )
        )

        self._join_btn = Button(description="Join",
                                icon="compress",
                                layout=Layout(width='auto'))

        self._join_btn.on_click(on_join_btn_clicked)

        self.btn_right_menu_enabled = ToggleButton(
            description="Menu",
            tooltip="Disable right menu for a better navigation experience.",
            icon="eye-slash",
            disabled=False,
            value=not self._bbox_state.right_menu_enabled,
            layout=Layout(width="70px")
        )

        self.btn_right_menu_enabled.observe(
            self.on_right_menu_enabled_clicked,
            "value"
        )

        self.footer = HBox(
            [
                self._navi,
                self._save_btn,
                self._undo_btn,
                self._redo_btn,
                self._join_btn,
                self.btn_right_menu_enabled,
            ],
            layout=Layout(
                display='flex',
                flex_flow='row wrap',
                align_items='center'
            )
        )

        self._app_state.index = 0

        self._image_box.state.subscribe(self.render_right_menu, 'bbox_coords')
        if self._image_box.state.bbox_coords:
            self.render_right_menu(self._image_box.state.bbox_coords)

    def on_trajectory_enabled_clicked(self, change: dict):
        self._bbox_state.drawing_trajectory_enabled = not self._bbox_state.drawing_trajectory_enabled  # noqa: E501
        self._bbox_state.trajectory = {}
        self.refresh_gui()

    def on_right_menu_enabled_clicked(self, change: dict):
        menu_enabled = self._bbox_state.right_menu_enabled
        self._bbox_state.right_menu_enabled = not menu_enabled
        self.refresh_gui()

    def refresh_gui(self):
        self._image_box.clear_layer(-1)
        self.clear_right_menu()
        self.render_right_menu(self._image_box.state.bbox_coords)
        # refreshing bbox_coords on canvas
        self._image_box.state.bbox_coords = self._image_box.state.bbox_coords

    def render_right_menu(self, bbox_coords: List[BboxVideoCoordinate]):
        if self.on_bbox_drawn:
            self.on_bbox_drawn(bbox_coords)
            self.right_menu._render(bbox_coords, self._bbox_state.labels)
        if self._bbox_state.drawing_trajectory_enabled:
            self.draw_trajectory(bbox_coords)

    def clear_right_menu(self):
        self.right_menu._bbox_list.clear()
        self.right_menu.children = []

    def _redo_clicked(self, event: dict):
        if self.history.label is not None:
            self._bbox_state.labels.append(self.history.label)
            self.history.label = []
        if self.history.trajectory:
            for k, v in self.history.trajectory.items():
                self._bbox_state.trajectory[k] = v
            self.history.trajectory = {}
        if self.history.bbox_coord:
            tmp_bbox_coords = deepcopy(self._image_box.state.bbox_coords)
            tmp_bbox_coords.append(self.history.bbox_coord)
            self._image_box.state.bbox_coords = tmp_bbox_coords
            self.history.bbox_coord = None

    def _undo_clicked(self, event: dict):
        if self._bbox_state.labels:
            tmp_labels = deepcopy(self._bbox_state.labels)
            self.history.label = tmp_labels.pop()
            self._bbox_state.labels = tmp_labels

        tmp_bbox_coords = None
        if self._image_box.state.bbox_coords:
            tmp_bbox_coords = deepcopy(self._image_box.state.bbox_coords)
            self.history.bbox_coord = tmp_bbox_coords.pop()
            self._image_box.state.bbox_coords = tmp_bbox_coords

        if self.history.bbox_coord and self.history.bbox_coord.id in self._bbox_state.trajectory:
            tmp_trajectory = deepcopy(self._bbox_state.trajectory)
            self.history.trajectory = {
                self.history.bbox_coord.id: tmp_trajectory[self.history.bbox_coord.id]
            }
            del tmp_trajectory[self.history.bbox_coord.id]
            self._bbox_state.trajectory = tmp_trajectory
        self.refresh_gui()

    def view_idx_changed(self, index: int):
        # store the last bbox drawn before index update
        self._bbox_state.set_quietly(
            'coords',
            self._image_box._state.bbox_coords
        )
        self.clear_right_menu()
        self._app_state.index = index

    def draw_trajectory(self, bbox_coords: List[BboxVideoCoordinate]):
        """Draw trajectory checking if object lives on bbox canvas"""
        coords = self._bbox_video_to_trajectory(bbox_coords)
        coord_ids = [i.id for i in self._image_box.state.bbox_coords]
        for obj_id, value in coords.items():
            if len(value) > 1 and obj_id in coord_ids:
                self.bbox_trajectory.draw_trajectory(
                    self._image_box.multi_canvas[-1],
                    value,
                    self._image_box.state.image_scale
                )

    def _bbox_video_to_trajectory(
            self, coords: List[BboxVideoCoordinate]) -> Dict[str, List[BboxCoordinate]]:
        """Group objects and stores (in memory) a list of bbox coordinates"""
        def sort(k):
            return k['id']

        coords = [asdict(c) for c in coords]
        for k, v in groupby(sorted(coords, key=sort), sort):
            value = list(v)
            for i in value:
                bbox_coord = BboxCoordinate(*list(i.values())[:4])
                try:
                    if bbox_coord not in self._bbox_state.trajectory[k]:
                        self._bbox_state.trajectory[k].append(bbox_coord)
                except Exception:
                    self._bbox_state.trajectory[k] = [bbox_coord]

        return self._bbox_state.trajectory

In [ ]:
#exporti

class BBoxVideoAnnotatorController(BBoxAnnotatorController):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _update_coords(self, index: int):  # from annotations
        image_path = str(self._storage.images[index])
        coords = self._storage.get(image_path) or {}
        self._bbox_state.labels = coords.get('labels', [])
        self._bbox_state.coords = [BboxVideoCoordinate(**c) for c in coords.get('bbox', [])]

    def _save_annotations(self, index: int, *args, **kwargs):
        image_path = str(self._storage.images[index])
        self._storage[image_path] = {
            'bbox': [asdict(bbox) for bbox in self._bbox_state.coords],
            'labels': self._bbox_state.labels,
        }
        self._storage.save()

    def update_storage_labels(self, change: dict, index: int):
        """Receive an object label update and updates all
        object's labels that share the same id"""
        self._bbox_state.labels[index] = [change['new']]
        bbox_coord_id = self._bbox_state.coords[index].id
        for image_path, bbox_or_labels in self._storage.items():
            if not bbox_or_labels or 'bbox' not in bbox_or_labels:
                break
            for i, bbox_coord in enumerate(bbox_or_labels['bbox']):
                if bbox_coord['id'] == bbox_coord_id:
                    self._storage[image_path]['labels'][i] = [change['new']]

    def update_storage_id(self, merged_ids: List[str]):
        """Update objects id's once one of them are merged.
        Returns the merged ids trajectory."""
        merge_id = "-".join(merged_ids)

        for image_path, bbox_or_labels in self._storage.items():
            if not bbox_or_labels or 'bbox' not in bbox_or_labels:
                break
            for bbox_coord in bbox_or_labels['bbox']:
                if bbox_coord['id'] in merged_ids:
                    bbox_coord['id'] = merge_id

    def _idx_changed(self, index: int):
        """
        On index change save an old state and update
        current image path and bbox coordinates for
        visualisation
        """
        self._save_annotations(self._last_index)
        self._update_im(index)
        self._update_coords(index)
        self._last_index = index

    def sync_labels(self, bbox_coords: List[BboxVideoCoordinate]):
        """Update labels according to the past frame labels
        or add an empty label to the bbox state. It also avoid to
        add empty labels if its length is the same as the bbox_coords"""
        num_classes = len(self._bbox_state.labels)

        for i, _ in enumerate(bbox_coords[num_classes:], num_classes):
            try:
                image_path = self._storage.images[self._app_state.index - 1]
                coords = self._storage.get(str(image_path)) or {}
                self._bbox_state.labels.append(
                    coords.get('labels')[i]
                )
            except Exception:
                self._bbox_state.labels.append([])

In [ ]:
#export

class BBoxVideoAnnotator(BBoxAnnotator):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **{**kwargs, 'render_previous_coords': False})
        pub.unsubscribe(self.controller._idx_changed, f'{self.app_state.root_topic}.index')
        pub.unsubAll(f'{self.app_state.root_topic}.index')
        state_params = {**self.bbox_state.dict()}
        state_params.pop('_uuid')
        self.bbox_state = BBoxVideoState(
            uuid=self.bbox_state._uuid,
            **state_params
        )

        self.controller = BBoxVideoAnnotatorController(
            app_state=self.app_state,
            bbox_state=self.bbox_state,
            storage=self.storage
        )

        self.view = BBoxAnnotatorVideoGUI(
            app_state=self.app_state,
            bbox_state=self.bbox_state,
            on_save_btn_clicked=self.on_save_btn_clicked,
            drawing_enabled=self._output_item.drawing_enabled,
            on_label_changed=self.update_labels,
            on_join_btn_clicked=self.merge_tracks_selected,
            on_bbox_drawn=self.controller.sync_labels
        )

    def update_labels(self, change: dict, index: int):
        """Saves bbox_canvas_state coordinates data
        on bbox_state and save all on storage."""
        self.bbox_state.set_quietly('coords', self.view._image_box._state.bbox_coords)
        self.controller.update_storage_labels(change, index)

    def on_save_btn_clicked(self, bbox_coords: List[dict]):
        self.controller.save_current_annotations(bbox_coords)

    def _update_state_id(self, merged_ids: List[str], bbox_coords: List[BboxVideoCoordinate]):
        merged_id = "-".join(merged_ids)

        for i, coord in enumerate(bbox_coords):
            if merged_id and bbox_coords[i].id in merged_ids:
                bbox_coords[i].id = merged_id

    def merge_tracks_selected(self, change: dict):
        selecteds = self.bbox_state.bbox_coords_selected

        if selecteds:
            merged_ids = [selected.bbox_video_coordinate.id for selected in selecteds]

            self._update_state_id(
                merged_ids=merged_ids,
                bbox_coords=self.view._image_box.state.bbox_coords
            )

            self.controller.update_storage_id(
                merged_ids=merged_ids
            )

            # merge trajectories
            key = "-".join(merged_ids)
            tmp_trajectory = self.bbox_state.trajectory.copy()
            for id, bbx in tmp_trajectory.items():
                if id in merged_ids:
                    try:
                        self.bbox_state.trajectory[key] += bbx
                    except Exception:
                        self.bbox_state.trajectory[key] = bbx

                    if id in self.bbox_state.trajectory:
                        # delete old trajectory id
                        del self.bbox_state.trajectory[id]

            # finished merge cleans selected bbox coords
            self.bbox_state.bbox_coords_selected = []

            self.view.clear_right_menu()
            self.view.render_right_menu(self.view._image_box.state.bbox_coords)

In [ ]:
#hide
from ipyannotator.mltypes import InputImage, OutputVideoBbox

in_p = InputImage(image_dir='pics', image_width=200, image_height=200)
out_p = OutputVideoBbox(classes=['Label 01', 'Label 02'])

In [ ]:
#hide
! rm -rf ../data/projects/bbox_video/results

In [ ]:
# hide
from ipyannotator.storage import construct_annotation_path
from ipyannotator.datasets.generators import create_mot_ds

project_path = Path('../data/projects/bbox_video')

anno_file_path = construct_annotation_path(project_path)

create_mot_ds(project_path, 'pics', 4)

In [ ]:
# hide
bb = BBoxVideoAnnotator(
    project_path=Path(project_path),
    input_item=in_p,
    output_item=out_p,
    annotation_file_path=anno_file_path
)

bb

In [ ]:
from ipyannotator.storage import construct_annotation_path


def delete_result_file():
    ! rm -rf ../data/projects/bbox_video/results


@pytest.fixture
def bbox_video_fixture():
    delete_result_file()
    project_path = Path('../data/projects/bbox_video')
    anno_file_path = construct_annotation_path(project_path)
    in_p = InputImage(image_dir='pics', image_width=640, image_height=400)
    out_p = OutputVideoBbox(classes=['Label 01', 'Label 02'])

    return BBoxVideoAnnotator(
        project_path=project_path,
        input_item=in_p,
        output_item=out_p,
        annotation_file_path=anno_file_path
    )


@pytest.fixture
def trajectory_fixture(bbox_video_fixture):
    fixture = bbox_video_fixture

    coords = [
        BboxVideoCoordinate(x=371, y=405, width=81, height=249, id='0'),
        BboxVideoCoordinate(x=677, y=186, width=78, height=171, id='1')
    ]

    fixture.view._image_box._state.bbox_coords = coords

    fixture.view._navi._next_btn.click()

    coords = [
        BboxVideoCoordinate(x=374, y=408, width=90, height=189, id='0'),
        BboxVideoCoordinate(x=686, y=189, width=75, height=135, id='1')
    ]

    fixture.view._image_box._state.bbox_coords = coords

    fixture.view._navi._next_btn.click()

    fixture.view._navi._prev_btn.click()
    fixture.view._navi._prev_btn.click()

    assert fixture.app_state.index == 0
    assert len(fixture.view.right_menu.children) == 2

    return fixture

In [ ]:
%%ipytest
def test_it_keeps_checkboxes_alive_on_user_navigation(trajectory_fixture):
    # select objects to join
    trajectory_fixture.view.right_menu[0].object_checkbox.value = True
    assert trajectory_fixture.view.right_menu[1].object_checkbox.value == False

    #next page
    trajectory_fixture.view._navi._next_btn.click()
    assert trajectory_fixture.view.right_menu[0].object_checkbox.value == True
    assert trajectory_fixture.view.right_menu[1].object_checkbox.value == False
    
    #previous page
    trajectory_fixture.view._navi._prev_btn.click()
    assert trajectory_fixture.view.right_menu[0].object_checkbox.value == True
    assert trajectory_fixture.view.right_menu[1].object_checkbox.value == False

In [ ]:
%%ipytest
def test_it_clear_right_menu_on_join_btn_click(bbox_video_fixture):
    for i in range(4):
        if not bbox_video_fixture.bbox_state.coords:
            coords=[
                BboxVideoCoordinate(x=371+i*10, y=405+i*10, width=81, height=249, id='0'), 
                BboxVideoCoordinate(x=677+i*10, y=186+i*10, width=78, height=171, id='1')
            ]

            bbox_video_fixture.bbox_state.coords = coords
        
        # alternate selects between the first and second element from the right menu
        elem = 1 if i % 2 != 0 else 0
        bbox_video_fixture.view.right_menu[elem].object_checkbox.value = True
        bbox_video_fixture.view._navi._next_btn.click()

    #click on join
    bbox_video_fixture.view._join_btn.click()

    assert bbox_video_fixture.bbox_state.bbox_coords_selected == []
    
    for i in range(4):
        # select the first element from the right menu
        elem = 1 if i % 2 != 0 else 0
        assert bbox_video_fixture.view.right_menu[elem].object_checkbox.description == '0-1'

In [ ]:
%%ipytest
def test_it_update_state_and_storage_on_label_change(trajectory_fixture):
    cls = out_p.classes[0]
#     bbox_video_fixture.view.right_menu[elem].object_checkbox.description == '0-1-0-1'
    trajectory_fixture.view.right_menu[0].dropdown_classes.value = cls
    assert trajectory_fixture.view.right_menu[1].dropdown_classes.value == None
    assert trajectory_fixture.bbox_state.labels == [[cls], []]
    trajectory_fixture.view._navi._next_btn.click()
    assert trajectory_fixture.view.right_menu[0].dropdown_classes.value == cls
    assert trajectory_fixture.view.right_menu[1].dropdown_classes.value == None

In [ ]:
%%ipytest
def test_it_can_draw_object_on_canvas_and_update_labels(bbox_video_fixture):
    bbox_video_fixture.view._navi._next_btn.click()
    coords = [
        BboxVideoCoordinate(x=371, y=405, width=81, height=249, id='0'),
        BboxVideoCoordinate(x=211, y=325, width=97, height=97, id='1')
    ]
    bbox_video_fixture.view._image_box._state.bbox_coords = coords
    bbox_video_fixture.view._navi._prev_btn.click()
    coords = [
        BboxVideoCoordinate(x=361, y=395, width=71, height=239, id='0'),
    ]
    bbox_video_fixture.view._image_box._state.bbox_coords = coords
    bbox_video_fixture.view.right_menu[0].dropdown_classes.value = out_p.classes[0]
    for k,v in bbox_video_fixture.storage.items():
        if v is not None:
            assert v['labels'][0][0] == out_p.classes[0]

In [ ]:
%%ipytest
def test_it_can_undo_trajectory_on_undo_click(trajectory_fixture):
    trajectory = {
        '0': [
            BboxCoordinate(x=371, y=405, width=81, height=249), 
            BboxCoordinate(x=374, y=408, width=90, height=189),
        ], 
        '1': [
            BboxCoordinate(x=677, y=186, width=78, height=171), 
            BboxCoordinate(x=686, y=189, width=75, height=135),
        ]
    }
    assert trajectory_fixture.bbox_state.trajectory == trajectory
    trajectory_fixture.view._undo_btn.click()
    assert trajectory_fixture.bbox_state.trajectory == {
        '0': [
            BboxCoordinate(x=371, y=405, width=81, height=249), 
            BboxCoordinate(x=374, y=408, width=90, height=189),
        ]
    }
    assert trajectory_fixture.view.history.trajectory == {
        '1': [
            BboxCoordinate(x=677, y=186, width=78, height=171), 
            BboxCoordinate(x=686, y=189, width=75, height=135),
        ]
    }

In [ ]:
%%ipytest
def test_it_can_undo_labels_on_undo_click(trajectory_fixture):
    labels = [[], []]
    assert trajectory_fixture.bbox_state.labels == labels
    trajectory_fixture.view._undo_btn.click()
    assert trajectory_fixture.bbox_state.labels == [[]]
    assert trajectory_fixture.view.history.label == []

In [ ]:
%%ipytest
def test_it_can_redo_trajectory_on_redo_click(trajectory_fixture):
    trajectory = {
        '0': [
            BboxCoordinate(x=371, y=405, width=81, height=249), 
            BboxCoordinate(x=374, y=408, width=90, height=189),
        ], 
        '1': [
            BboxCoordinate(x=677, y=186, width=78, height=171), 
            BboxCoordinate(x=686, y=189, width=75, height=135),
        ]
    }
    trajectory_fixture.view._undo_btn.click()
    trajectory_fixture.view._redo_btn.click()
    assert trajectory_fixture.view.history.trajectory == {}

In [ ]:
%%ipytest
def test_it_can_redo_labels_on_redo_click(trajectory_fixture):
    labels = [[], []]
    assert trajectory_fixture.bbox_state.labels == labels
    trajectory_fixture.view._undo_btn.click()
    assert trajectory_fixture.bbox_state.labels == [[]]
    trajectory_fixture.view._redo_btn.click()
    assert trajectory_fixture.bbox_state.labels == labels

In [ ]:
%%ipytest
def test_it_change_only_selected_checkboxes_on_join_trajectories(trajectory_fixture):
    trajectory_fixture.view.right_menu[1].object_checkbox.value = True
    
    assert len(trajectory_fixture.bbox_state.bbox_coords_selected) == 1
    
    trajectory_fixture.view._join_btn.click()
    
    inp1 = trajectory_fixture.view.right_menu[0].object_checkbox
    inp2 = trajectory_fixture.view.right_menu[1].object_checkbox
    
    assert inp1.description != inp2.description

In [ ]:
%%ipytest
def test_it_merge_storage_trajectories_on_join_trajectories(trajectory_fixture):
    assert trajectory_fixture.app_state.index == 0
    
    second_frame_coords = None
    for i in range(3):
        if i == 0:
            # select object 0 on the first frame
            trajectory_fixture.view.right_menu[0].object_checkbox.value = True
        elif i == 1:
            second_frame_coords = trajectory_fixture.view._image_box._state.bbox_coords[0]
        
        trajectory_fixture.view._navi._next_btn.click()

    coords = [
        BboxVideoCoordinate(x=400, y=455, width=111, height=298, id='0'), 
        BboxVideoCoordinate(x=722, y=210, width=90, height=190, id='1')
    ]
    trajectory_fixture.view._image_box._state.bbox_coords = coords

    # select object 0 on the third (current) frame
    trajectory_fixture.view.right_menu[0].object_checkbox.value = True
    # join the first and third frame
    trajectory_fixture.view._join_btn.click()
    # makes sure that the second frame is in the trajectory
    exists = False
    
    for k, trajectory in trajectory_fixture.bbox_state.trajectory.items():
        if second_frame_coords.bbox_coord() in trajectory:
            exists = True
    
    assert exists == True

In [ ]:
%%ipytest
def test_it_render_object_label_on_drawing(bbox_video_fixture):
    bbox_video_fixture.view._image_box._state.bbox_coords = [
        BboxVideoCoordinate(x=361, y=395, width=71, height=239, id='0'),
    ]
    cls = out_p.classes[0]
    bbox_video_fixture.view.right_menu[0].dropdown_classes.value = cls
    bbox_video_fixture.view._navi._next_btn.click()
    assert bbox_video_fixture.view._image_box._state.bbox_coords == []
    bbox_video_fixture.view._image_box._state.bbox_coords = [
        BboxVideoCoordinate(x=371, y=405, width=81, height=249, id='0'),
    ]
    assert bbox_video_fixture.view.right_menu[0].dropdown_classes.value == cls

In [ ]:
%%ipytest
def test_it_can_disable_drawing_trajectory(bbox_video_fixture):
    bbox_video_fixture.view.right_menu.trajectory_enabled_checkbox.value = False
    bbox_video_fixture.view._image_box._state.bbox_coords = [
        BboxVideoCoordinate(x=361, y=395, width=71, height=239, id='0'),
    ]
    bbox_video_fixture.view._navi._next_btn.click()
    bbox_video_fixture.view._image_box._state.bbox_coords = [
        BboxVideoCoordinate(x=371, y=405, width=81, height=249, id='0'),
    ]
    assert bbox_video_fixture.bbox_state.trajectory == {}

In [ ]:
%%ipytest
def test_it_can_navigate_on_disabled_drawing_trajectory(trajectory_fixture):
    trajectory_fixture.view._navi._next_btn.click()
    trajectory_fixture.view.right_menu.trajectory_enabled_checkbox.value = False
    trajectory_fixture.view._navi._prev_btn.click()
    assert trajectory_fixture.bbox_state.trajectory == {}
    trajectory_fixture.view.right_menu.trajectory_enabled_checkbox.value = True
    trajectory_fixture.view._navi._next_btn.click()
    assert trajectory_fixture.bbox_state.trajectory != {}

In [ ]:
%%ipytest
def test_it_sticky_object_selection_across_frames(trajectory_fixture):
    trajectory_fixture.view.right_menu[0].object_checkbox.value = True
    trajectory_fixture.view._navi._next_btn.click()
    assert trajectory_fixture.view.right_menu[0].object_checkbox.value == True

In [ ]:
%%ipytest
def test_it_render_right_menu_on_init(trajectory_fixture, bbox_video_fixture):
    assert len(bbox_video_fixture.storage) > 0
    assert len(bbox_video_fixture.view.right_menu.children[1].children) > 0

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()